In [ ]:
#import the necessary libraries

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#display the plots inside this environment without having to call show()
#set the dataframe options - should show all columns

%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

# Exploratory Analysis

> In this section I go through the data with an intention of finding out the structure, distribution and general feel of the data.
>
> The exploration will be guided by both the data, as well as intuition. For example, I simply do not expect an age of over (say) 150 years - this would mean that the data is not very accurate.

## Gather

In [ ]:
#load the data onto an on-memory dataframe
df = pd.read_csv('Nutrition__Physical_Activity__and_Obesity_-_Behavioral_Risk_Factor_Surveillance_System.csv')

## Assess

> While assessing the data, I will majorly concentrate on highlighting and correcting data quality issues.
>
> Any characteristic of the data that might help later in in the analysis will also be noted.

In [ ]:
#show five random records from the dataset
df.sample(5)

In [ ]:
#print the size of the data
df.shape

> The dataset has 53392 records and 33 dimensions as represented in the cell above.

In [ ]:
#display the columns that the dataset has
df.columns

In [ ]:
#a summary of the numerical columns
df.describe().transpose()

> From the summary it is evident that the study was carried out between 2011 and 2016, as shown by the rangef of YearStart and YearEnd.

> An important attribute is how much null values there are in the dataset.
>
> In the cell below, I look at how many null values exist in each column

In [ ]:
# display the null values in each column
df.isnull().sum().sort_values()

> Columns that have only one unique value can as well be excluded since they do not add much information to the data.
>
> In the cell below I probe each column to see the number of unique values it contains

In [ ]:
#show the number of unique values each column has
for col in df.columns:
    print(f'{col} has {df[col].nunique()} unique values')

> It is also important to get to know the relationships that exist between the columns early on in the analysis

In [ ]:
#display the relationship of each column against every other column
sns.pairplot(df);

> (YearStart, YearEnd) and (Data_Value, Data_Value_Alt) have perfect correlation to each other. This means that they are essentially the same data replicated across multiple columns. One of them can be dropped

In [ ]:
#find out the number of duplicated records
df.duplicated().sum()

> All records in the dataset are unique as shown in the cell above

> It is also necessary to understand the data types of the values for each column.
>
> This is important to know since not all operations can be performed on all data types

In [ ]:
df.dtypes

### Issues with data

> After some exploration I realized the data has some quality issues, which I will focus on in this section
>
> By the end of this section I hope that I will have clean and structured data

#### Data tidiness

> In this subsection, issues of atomicity of data will be higlighted.
>
> To make for easy analysis, the following guidelines should be adhered to;
>
>1. Each variable should have its own column
>2. Each observation should be a row (no duplicates should exist)
>3. Each organizational unit should be a table/dataset

> The intention is to separate each dataset on its own, then later bring it together in an organized manner. For example, it does not make for good viewing that location alone is represented with 4 different columns in the same dataset.

1. The dataset has a semi-dataset that represents questions
2. The dataset has a semi-dataset that represents topics
3. The dataset has a semi-dataset that represents classes
4. The dataset has a semi-dataset that represents locations
5. The dataset has a semi-dataset that represents stratifications
6. YearStart has a perfect correlation to YearEnd - this could mean that they contain the same values, which then makes one of the columns redundant
7. Data_Value has a perfect correlation to Data_Value_Alt

#### Data Cleanliness

> Columns which have less than 2 unique values really do not add much to our analysis
>
> All records will most likely have the same value, and so the column can be ignored

1. Datasource has 1 unique value
2. Data_Value_Unit has 0 unique values - this is due to the fact that the column does not have a single non-null value. It shall be handled as a missing data issue.
3. Data_Value_Type has 1 unique value
4. Data_Value_Footnote_Symbol has 1 unique value
5. Total has 1 unique values

#### Missing Data

> Missing values could mean that the data is improperly stored, or simply that such data was not captured.
>
> In this section I highlight issues with missing data.

1. GeoLocation has 1008 missing values
2. Data_Value has 5046 missing values
3. Sample_Size has 5046 missing values
4. High_Confidence_Limit has 5046 missing values
5. Low_Confidence_Limit has 5046 missing values
6. Data_Value_Alt has 5046 missing values
7. Race/Ethnicity has 38136 missing values
8. Income has 40043 missing values
9. Age(years) has 41954 missing values
10. Education has 45764 missing values
11. Data_Value_Footnote has 48346 missing values
12. Data_Value_Footnote_Symbol has 48346 missing values
13. Gender has 49578 missing values
14. Total has 51485 missing values
15. Data_Value_Unit has 53392 missing values

### Cleaning data

> In this section I address the issues outlined above.
>
> This step will be repetitive, to try and obtain clean and tidy data
>
> I will detail the problem that I want solved, propose a resolution plan and then implement the plan. As a final step I will test to see that my expectations have been achieved.

##### Problem Definition

The dataset has a semi-dataset that represents questions

##### Resolution Plan

Create a dataset that contains only information about questions

##### Implementation and Testing

In [ ]:
questions_df = df[['QuestionID', 'Question']].drop_duplicates()
questions_df.reset_index(drop=True, inplace=True)
questions_df.head()

> We now have an atomic dataset representing questions alone
>
>We can use the QuestionID to reference a question in a different table if need be

In [ ]:
questions_df.shape

> In total, the the data contains answers to 9 questions, which are listed below
>
> When cleaning the main table, I will drop the Question description since it is long. However, it a question can still be referenced using the ID

In [ ]:
questions_df

##### Problem Definition

The dataset has a semi-dataset that represents topics

##### Resolution plan

Create a dataset that contains only information about topics

##### Implementation and Testing

In [ ]:
topics_df = df[['TopicID', 'Topic']].drop_duplicates()
topics_df.reset_index(drop=True, inplace=True)
topics_df.head()

In [ ]:
topics_df.shape

> Now topics have their own table, and can be referenced using the ID - which can remain in the main table as a foreign key

##### Problem Definition

The dataset has a semi-dataset that represents classes

##### Resolution Plan

Create a dataset that contains only information about classes

##### Implementation and Testing

In [ ]:
classes_df = df[['ClassID', 'Class']].drop_duplicates()
classes_df.reset_index(drop=True, inplace=True)
classes_df.head()

In [ ]:
classes_df.shape

> It is becoming clear that classes and topics could be very related.
>
> This means that our initial dataset had 4 columns(Class, ClassID, Topic and TopicID) representing the same dimension.

In [ ]:
# df[((df['Class'] == 'Obesity/Weight Status') and (df['Topic'] == 'Obesity/Weight Status'))
#    or ((df['Class'] == 'Fruits and Vegetables - Behavior') and (df['Topic'] == 'Fruits and Vegentables'))
#   or ((df['Class'] == 'Physical Activity - Behavior') and (df['Topic'] == 'Physical Activity'))].sum()

##### Problem Definition

The dataset has a semi-dataset that represents locations

##### Resolution Plan

Create a dataset that contains only information about location

##### Implementation and Testing

In [ ]:
location_df = df[['LocationID', 'LocationAbbr', 'GeoLocation', 'LocationDesc']].drop_duplicates()
location_df.reset_index(drop=True, inplace=True)
location_df.head()

In [ ]:
location_df.shape

> The data containing information about different locations is now in its own dataframe. Redundant columns can now be dropped from the main dataframe

##### Problem Definition

The dataset contains a semi-dataset the represents information on how stratification was done

##### Resolution Plan

Create a dataset that contains only information about stratification

##### Implementation and Testing

In [ ]:
stratification_df = df[['Stratification1', 'StratificationID1']].drop_duplicates()
stratification_df.reset_index(drop=True, inplace=True)

categories_df = df[['StratificationCategory1', 'StratificationCategoryId1']].drop_duplicates()
categories_df.reset_index(drop=True, inplace=True)

In [ ]:
stratification_df.head()

In [ ]:
categories_df.head()

In [ ]:
stratification_df.shape

In [ ]:
categories_df.shape

In [ ]:
categories_df

> The data containing information about categories is now in their own dataframes. Redundant columns can now be dropped from the main dataframe

##### Problem Definition

Some columns in the dataset have less than 2 unique values

##### Resolution Plan

Exclude from the dataset columns which have at most 1 unique value

##### Implementation and Testing

In [ ]:
df_clean = df.drop(['Datasource', 'Data_Value_Unit', 'Data_Value_Type', 'Data_Value_Footnote_Symbol', 'Total', 'DataValueTypeID'], axis=1)
df_clean.sample(4)

In [ ]:
df_clean.shape

In [ ]:
for col in df_clean.columns:
    print(f'{col} has {df_clean[col].nunique()} unique values')

> The main dataframe now only contains columns which can be used for some type of grouping, and this can improve the analysis

##### Problem Definition

The dataset contains information that describes different observations

##### Resolution Plan

Exclude information that describe location, categories, classes, topics and questions - only leave IDs which can be used for referencing if there is need

##### Implementation and Testing

In [ ]:
atomic_df = df_clean.drop(['LocationID', 'LocationDesc', 'GeoLocation', 'StratificationID1', 'StratificationCategoryId1', 'Class', 'Topic', 'Question', 'TopicID'], axis=1)
atomic_df.sample(4)

In [ ]:
atomic_df.shape

> The atomic dataframe now obeys one of the laws of atomicity - each variable is represented by one column. A direct result of this is the sharp reduction in the dimensions of the dataset

##### Problem Definition

YearStart has a perfect correlation to YearEnd

##### Resolution Plan

Drop one of YearStart and YearEnd, then rename the remaining column as simply Year

##### Implementation and Testing

In [ ]:
atomic_df = atomic_df.drop(['YearEnd'], axis=1)
atomic_df.rename(columns={'YearStart': 'Year'}, inplace=True)
atomic_df.head()

> This result has also resulted in the reduction of 1 dimension

##### Problem Definition

Data_Value has a perfect correlation to Data_Value_Alt

##### Resolution Plan

Drop one of Data_Value and Data_Value_Alt

##### Implementation and Testing

In [ ]:
atomic_df = atomic_df.drop(['Data_Value_Alt'], axis=1)
atomic_df.head()

In [ ]:
atomic_df.shape

> 1 unnecessary dimension has also been removed from the data

In [ ]:
atomic_df.isnull().sum().sort_values()

In [ ]:
atomic_df.dtypes

In [ ]:
atomic_df['Data_Value_Footnote'].unique()

In [ ]:
atomic_df['Data_Value_Footnote'].value_counts()

> Looking at the stratification column, it seems as though information about Age, Income, Race and Gender are encoded in long form.
>
> It now begins to make sense why such important indicators have numerous null values

In [ ]:
atomic_df['StratificationCategory1'].unique()

> It is also worth noting that the the values encoded in the StratificationCategory1 column are exactly the same columns that had so many null values
>
> I will extract each stratitification category individually and perform analysis independently, since it seems that the dataset is just a concatenation of other smaller datasets.

#### Education Dataset

> I begin by isolating data that was stratified by education

In [ ]:
education_df = atomic_df[atomic_df['StratificationCategory1'] == 'Education']
education_df.reset_index(drop=True, inplace=True)
education_df.head()

In [ ]:
education_df.shape

In [ ]:
education_df.isnull().sum().sort_values()

> The columns 'Age(years)', 'Income', 'Race/Ethnicity', 'Data_Value_Footnote' and 'Gender' have no use remaining in the dataset, since they contain 100% null values
>
> Also, the stratification columns can be ignored since they contain the same information

In [ ]:
education_df = education_df.drop(['Age(years)', 'Income', 'Race/Ethnicity', 'Data_Value_Footnote', 'Gender', 'StratificationCategory1', 'Stratification1'], axis=1)
education_df.head()

In [ ]:
education_df.groupby(['QuestionID', 'Education'])['Data_Value'].describe().transpose()

In [ ]:
education_df['Education'].unique()

In [ ]:
education_encoding = {'Education': {'Less than high school': 0, 'High school graduate': 1, 'Some college or technical school': 2, 'College graduate': 3}}
education_df = education_df.replace(education_encoding)
education_df.head()

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(20, 18), sharey=True, sharex=True)
fig.suptitle('Classification by Education');

k = 0
for i in range(3):
    if k >= questions_df.shape[0]:
        break
    for j in range(3):
        if k >= questions_df.shape[0]:
            break
        question_id = questions_df.iloc[[k]]['QuestionID'][k]
        sns.barplot(ax=axes[i][j], data=education_df[education_df['QuestionID']==question_id], x='Education', y='Data_Value', color='grey');
        axes[i][j].set_title(question_id);
        k += 1

> College graduates reported the least percentage of people who had obesity, while those with less than high education reported the highest percentages of people with obesity
>
> However, there seemed to be no clear disparity among people of the different levels of education by their overweight classification. In fact, college graduates reported slighlty higher percentages of overweight status.
>
> It is somewhat confounding that more college graduates confessed to be consuming fruits less than once daily than those from all the other levels of education. On the flipside, around 30% of people with less than high school education revealed that they consume veges less than once daily - almost twice the percentage of college graduates who reported to consume veges less than once daily.
>
> People with at least a college degree are also more likely to engage in moderate intensity or vigorous-intensity aerobics, although those with less than high school education were more likely to engage in leisure-time physical activity.

In [ ]:
sns.pairplot(education_df, hue='Education');

> Data values do not show any correlation to education level. Values for all education levels occur all across the spectrum.

In [ ]:
plt.figure(figsize=(20, 15));
sns.boxplot(x="QuestionID", y="Data_Value", data=education_df, hue='Education', whis=[0, 100], width=.6)
plt.title('\nHow repondents performed on different questions based on education\n');
plt.xlabel('Questions');
plt.ylabel('Values');

> Q043 witnessed the widest range of values, denoting the big difference in responses. On the other hand, values for Q037 were almost similar across the different 
education levels

In [ ]:
questions_df[questions_df['QuestionID'] == 'Q043']

In [ ]:
questions_df[questions_df['QuestionID'] == 'Q037']

#### Stratitification by Age

In [ ]:
age_df = atomic_df[atomic_df['StratificationCategory1'] == 'Age (years)']
age_df.reset_index(drop=True, inplace=True)
age_df.head()

In [ ]:
age_df.shape

In [ ]:
age_df.isnull().sum().sort_values()

> As with the education dataframe, the other stratification columns need to be removed since they do not contain any information.

In [ ]:
age_df = age_df.drop(['Education', 'Income', 'Race/Ethnicity', 'Data_Value_Footnote', 'Gender', 'StratificationCategory1', 'Stratification1'], axis=1)
age_df.head()

In [ ]:
#Showing 9 subplots side by side, one for each question in the questions dataset and examining how people of different ages respond to them
#Bar plots are generally used to compare categorical data - in this case, how responses to the questions vary by age groups

fig, axes = plt.subplots(3, 3, figsize=(20, 18), sharey=True, sharex=True) #define the figure and axis that will be used for plotting
fig.suptitle('Classification by Age');

k = 0 # I used this variable to ensure that the plot does not burst the size questions, and plot a non existent dimension. Doing so will definitely throw an error
for i in range(3): # Lopping over 3 times, for the 3 rows in the subplots matrix
    if k >= questions_df.shape[0]: # stop the looping if all questions have been encountered
        break
    for j in range(3): # loop over each column in the subplot matrix
        if k >= questions_df.shape[0]: # stop the looping if all questions have been encountered
            break
        question_id = questions_df.iloc[[k]]['QuestionID'][k] #retrieve the ID of the each question, and assign it to the varibale question_id
        sns.barplot(ax=axes[i][j], data=age_df[age_df['QuestionID']==question_id], x='Age(years)', y='Data_Value', color='grey'); # draw the bar plot
        axes[i][j].set_title(question_id); # set the title of the subplot as the question ID
        k += 1 # increment 1, to ensure that the computation does not sink into an infinite loop

> People in the mid-years reported more cases of obesity then people in the lower and upper edges. The observation was, however, reversed when the respondents were asked whether they engage in muscle-strenghtening activities at least twice a week. People from the lower edges and upper edges (with respect to age) reported higher percentages than those in the mid-years. Whether these two observations are directly correlated makes for a good research question.
>
> A higher proportion of lower age adults (18-24) reported to perform either moderate-intensity or vigorous-intensity aerobics at least twice a week. Rather unsurprisingly, the percentage of people in the '65 or older' category who reported to not participate in leisure time physical activity was higher than any other age-group.

In [ ]:
plt.figure(figsize=(20, 15));
sns.boxplot(x="QuestionID", y="Data_Value", data=age_df, hue='Age(years)', whis=[0, 100], width=.6)
plt.title('\nHow repondents performed on different questions based on age groups\n');
plt.xlabel('Questions');
plt.ylabel('Values');

> Question Q043 elicited almost similar responses from people in all the age groups.
>
> Conversely, a higher percentage of older people agreed to question Q047 than people in younger age groups

In [ ]:
questions_df[questions_df['QuestionID'] == 'Q043']

In [ ]:
questions_df[questions_df['QuestionID'] == 'Q047']

#### Classifiaction by Income

In [ ]:
income_df = atomic_df[atomic_df['StratificationCategory1'] == 'Income']
income_df.reset_index(drop=True, inplace=True)
income_df.head()

In [ ]:
income_df.shape

In [ ]:
income_df.isnull().sum().sort_values()

In [ ]:
income_df = income_df.drop(['Education', 'Age(years)', 'Race/Ethnicity', 'Data_Value_Footnote', 'Gender', 'StratificationCategory1', 'Stratification1'], axis=1)
income_df.head()

In [ ]:
income_df['Income'].unique()

In [ ]:
income_encoding = {'Income': {'Less than $15,000': 0, '$15,000 - $24,999': 1, '$25,000 - $34,999': 2, '$35,000 - $49,999': 3, '$50,000 - $74,999': 4,
                                   '$75,000 or greater': 5, 'Data not reported': 6}}
income_df = income_df.replace(income_encoding)
income_df.head()

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(20, 18), sharey=True, sharex=True)
fig.suptitle('Income classification');

k = 0
for i in range(3):
    if k >= questions_df.shape[0]:
        break
    for j in range(3):
        if k >= questions_df.shape[0]:
            break
        question_id = questions_df.iloc[[k]]['QuestionID'][k]
        sns.barplot(ax=axes[i][j], data=income_df[income_df['QuestionID']==question_id], x='Income', y='Data_Value', color='grey');
        axes[i][j].set_title(question_id);
        k += 1

> Over 30% of people in the lowest income band divluged that they consumed veges less than once daily. This is almost double the percentage of people in the  highest income band who also reported to consume veges less than once daily.
>
> The same pattern is seen when asked whether they participate in leisure time physical activities. Almost 40% of people in the lowest income band said they don't, compared to only 15% of people in the highest band who do not engage in leisure time physical activities.

In [ ]:
plt.figure(figsize=(20, 15));
sns.boxplot(x="QuestionID", y="Data_Value", data=income_df, hue='Income', whis=[0, 100], width=.6)
plt.title('\nHow repondents performed on different questions based on income\n');
plt.xlabel('Questions');
plt.ylabel('Values');

> Almost all but questions Q019 and Q047 returned similar percentages

In [ ]:
questions_df[questions_df['QuestionID'] == 'Q019']

In [ ]:
questions_df[questions_df['QuestionID'] == 'Q047']

#### Classification by Ethnicity

In [ ]:
race_df = atomic_df[atomic_df['StratificationCategory1'] == 'Race/Ethnicity']
race_df.reset_index(drop=True, inplace=True)
race_df.head()

In [ ]:
race_df.shape

In [ ]:
race_df.isnull().sum().sort_values()

In [ ]:
race_df = race_df.drop(['Education', 'Age(years)', 'Income', 'Data_Value_Footnote', 'Gender', 'StratificationCategory1', 'Stratification1'], axis=1)
race_df.head()

In [ ]:
race_df['Race/Ethnicity'].unique()

In [ ]:
race_encoding = {'Race/Ethnicity': {'Non-Hispanic White': 0, 'Non-Hispanic Black': 1, 'Hispanic': 2, 'Asian': 3, 'Hawaiian/Pacific Islander': 4,
                                   'American Indian/Alaska Native': 5, '2 or more races': 6, 'Other': 7}}
race_df = race_df.replace(race_encoding)
race_df.head()

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(20, 18), sharey=True, sharex=True)
fig.suptitle('Classification by Race and Ethnicity');

k = 0
for i in range(3):
    if k >= questions_df.shape[0]:
        break
    for j in range(3):
        if k >= questions_df.shape[0]:
            break
        question_id = questions_df.iloc[[k]]['QuestionID'][k]
        sns.barplot(ax=axes[i][j], data=race_df[race_df['QuestionID']==question_id], x='Race/Ethnicity', y='Data_Value', color='grey');
        axes[i][j].set_title(question_id);
        k += 1

> There was not not much disticntion between the participating ethnic groups, based on how they answered the questions.
>
> The only conspicuous difference can be seen in Asians, who reported an unbelievably low percentage of obesity cases compared to other ethnic groups.

In [ ]:
plt.figure(figsize=(20, 15));
sns.boxplot(x="QuestionID", y="Data_Value", data=race_df, hue='Race/Ethnicity', whis=[0, 100], width=.6)
plt.title('\nHow repondents performed on different questions based on race\n');
plt.xlabel('Questions');
plt.ylabel('Values');

> The participating races answered all the questions almost similarly. The only difference being question Q036 which had one ethnic group reporting a much lower percentage than the other ethnic groups.

In [ ]:
questions_df[questions_df['QuestionID'] == 'Q036']

In [ ]:
keys = list(race_encoding['Race/Ethnicity'].keys())
vals = list(race_encoding['Race/Ethnicity'].values())
print(keys[vals.index(3)])

> Asians reported remarkedly lower obesity percentages than the other ethnic groups

#### Classification by gender

In [ ]:
gender_df = atomic_df[atomic_df['StratificationCategory1'] == 'Gender']
gender_df.reset_index(drop=True, inplace=True)
gender_df.head()

In [ ]:
gender_df.dtypes

In [ ]:
gender_df.shape

In [ ]:
gender_df.isnull().sum().sort_values()

In [ ]:
gender_df = gender_df.drop(['Age(years)', 'Income', 'Race/Ethnicity', 'Data_Value_Footnote', 'Education', 'StratificationCategory1', 'Stratification1'], axis=1)
gender_df.head()

In [ ]:
plt.figure(figsize=(20, 15));
sns.boxplot(x="QuestionID", y="Data_Value", data=gender_df, hue='Gender', whis=[0, 100], width=.6, palette="vlag")
plt.title('\nHow repondents performed on different questions based on gender\n');
plt.xlabel('Questions');
plt.ylabel('Values');

> Question Q037 produced the most varied values for males and females.

In [ ]:
#The question in question is;

questions_df[questions_df['QuestionID'] == 'Q037']

In [ ]:
plt.figure(figsize=(20, 15));
sns.violinplot(x="QuestionID", y="Data_Value", hue="Gender", data=gender_df, palette="vlag", split=True);
plt.title('\nHow repondents performed on different questions based on gender\n');
plt.xlabel('Questions');
plt.ylabel('Values');

> The violin plots corroborates that the distribution plots for males and females on question Q037 is different. The rest of the distributions show some overlap, suggesting a little similarity

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(20, 18), sharey=True, sharex=True)
fig.suptitle('Gender classification');

k = 0
for i in range(3):
    if k >= questions_df.shape[0]:
        break
    for j in range(3):
        if k >= questions_df.shape[0]:
            break
        question_id = questions_df.iloc[[k]]['QuestionID'][k]
        sns.barplot(ax=axes[i][j], data=gender_df[gender_df['QuestionID']==question_id], x='Gender', y='Data_Value', color='grey');
        axes[i][j].set_title(question_id);
        k += 1

> A higher percentage of males was found to be overweight than females. However, a higher percentage of females than males disclosed that they consume fruit less than once daily.
>
> A higher percentage of males than females said they consume veges less than once daily. A higher percentage of males than females also reported to engage in muscle-strenghtening activities at least twice a week.